In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [2]:
def genObs(length, rr, key_length, startup_length, symbols):
    x = ''.join(np.random.choice(symbols, size=startup_length))
    for i in range(length - startup_length):
        x = x + rr[x[-key_length:]]
    return x

def genRR(symbols, key_length):
    prefixes = ['']
    for i in range(key_length):
        prefixes = [x + y for x in prefixes for y in symbols]
    symarr = np.array(list(symbols))
    return {x:np.random.choice(symarr) for x in prefixes}

def genSample(num, rr, length=20, startup_length=5):
    keylen = max([len(x) for x,_  in rr.items()])
    symarr = np.array(list(set(''.join([x for x,_  in rr.items()]))))
    return [genObs(length, rr, keylen, startup_length, symarr) for _ in range(num)]

def randomBatch(tensor, batchSize=64):
    if type(tensor) is tuple:
        ids = np.random.choice(range(tensor[0].shape[0]), batchSize)
        return (x[ids,] for x in tensor)
    ids = np.random.choice(range(tensor.shape[0]), batchSize)
    return tensor[ids,]

In [5]:
rr = genRR('01234567', 5)
train0 = genSample(10000, rr)
valid0 = genSample(2000, rr)
train = np.array([np.array([int(y) for y in x]) for x in train0])
valid = np.array([np.array([int(y) for y in x]) for x in valid0])

In [6]:
valid[:10]

array([[6, 0, 4, 6, 7, 4, 5, 4, 2, 2, 2, 1, 6, 2, 4, 2, 1, 3, 0, 6],
       [3, 3, 6, 1, 6, 6, 4, 7, 3, 5, 3, 5, 5, 4, 6, 1, 5, 0, 0, 4],
       [5, 7, 2, 5, 4, 4, 6, 2, 3, 4, 0, 2, 2, 0, 5, 7, 0, 5, 7, 7],
       [6, 7, 1, 7, 3, 5, 1, 1, 0, 5, 7, 6, 0, 4, 3, 5, 5, 3, 2, 5],
       [5, 0, 1, 0, 1, 6, 0, 5, 4, 7, 6, 4, 0, 6, 1, 2, 6, 0, 3, 0],
       [1, 2, 0, 2, 3, 7, 7, 2, 6, 4, 0, 0, 3, 5, 3, 1, 3, 4, 4, 7],
       [4, 3, 6, 5, 5, 1, 5, 2, 7, 4, 3, 1, 0, 1, 4, 1, 5, 5, 2, 1],
       [6, 0, 3, 4, 2, 3, 4, 5, 3, 2, 3, 1, 6, 6, 1, 3, 7, 6, 3, 6],
       [1, 7, 4, 0, 6, 2, 0, 3, 3, 4, 4, 2, 2, 4, 2, 7, 6, 2, 2, 3],
       [3, 5, 6, 7, 0, 2, 7, 6, 3, 5, 3, 6, 7, 1, 7, 2, 2, 6, 7, 5]])

In [16]:
SEQ_LEN = 20
CHARSET_SIZE = 8
RNN_SIZE = [50,50]
LEARNING_RATE = 1e-2

InnerCell = lambda n: tf.nn.rnn_cell.BasicRNNCell(num_units=n, activation=tf.nn.elu)
#InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.relu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int32)

tfX = tf.one_hot(tfi_x, CHARSET_SIZE, dtype=tf.float32)
tfY = tfX[:,1:,:]

rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE], state_is_tuple=True)
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0], activation=tf.nn.elu)

tfH, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfHOut = tf.layers.dense(tfH, CHARSET_SIZE)


with tf.name_scope('LOSS'):
    tfDiff = tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfHOut[:,:-1,:])
    tfLoss = tf.reduce_mean(tfDiff[:,-10:], name='TRAIN-LOSS')
    tfEqEw = tf.cast(tf.equal(tf.argmax(tf.nn.softmax(tfHOut[:,:-1,:],dim=2), axis=2), tf.argmax(tfY, axis=2)), dtype=tf.float32)
    tfAccMax = tf.reduce_mean(tf.reduce_min(tfEqEw[:,-5:], axis=1), name='ACCURACY-OBS')
    tfAccMean = tf.reduce_mean(tfEqEw[:,-5:], name='ACCURACY-SYM')
    tfTrain = tf.train.AdamOptimizer(LEARNING_RATE).minimize(tfLoss)

#tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.argmax(tf.nn.softmax(tfHOut[:,:-1,:],dim=2), axis=2)

#tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tfi_y, tfOutR), dtype=tf.float32))
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [17]:
batch_size = 1000
num_steps  = 5
num_epochs = 200

valid_batch = {tfi_x: valid}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini = randomBatch(train, batchSize=batch_size)
        train_batch = {tfi_x:mini}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv, ac, acm = tfs.run([tfLoss, tfAccMean, tfAccMax], feed_dict=valid_batch)
        
        #if i % 10 == 0 and i > 0:
        #    print(tfOutR.eval(feed_dict=valid_batch)[:10] - valid[:10,1:])
        
        print('Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.2f}\t\tAC:{5:1.3f} ({6:1.3f})'.format(i,t1-t0,l0,l1,lv,ac, acm))
    #valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (0.461 sec): loss changed from 2.16 to 2.04		VL:2.07		AC:0.169 (0.000)
Epoch 1 (0.452 sec): loss changed from 2.06 to 2.02		VL:2.06		AC:0.184 (0.000)
Epoch 2 (0.454 sec): loss changed from 2.06 to 2.02		VL:2.05		AC:0.187 (0.003)
Epoch 3 (0.456 sec): loss changed from 2.04 to 2.01		VL:2.05		AC:0.193 (0.000)
Epoch 4 (0.45 sec): loss changed from 2.04 to 2.0		VL:2.04		AC:0.199 (0.000)
Epoch 5 (0.46 sec): loss changed from 2.04 to 2.01		VL:2.03		AC:0.202 (0.001)
Epoch 6 (0.461 sec): loss changed from 2.04 to 2.0		VL:2.03		AC:0.213 (0.001)
Epoch 7 (0.455 sec): loss changed from 2.02 to 1.98		VL:2.02		AC:0.219 (0.001)
Epoch 8 (0.453 sec): loss changed from 2.02 to 1.98		VL:2.01		AC:0.230 (0.003)
Epoch 9 (0.449 sec): loss changed from 2.0 to 1.95		VL:2.00		AC:0.240 (0.003)
Epoch 10 (0.453 sec): loss changed from 1.99 to 1.93		VL:1.98		AC:0.253 (0.007)
Epoch 11 (0.548 sec): loss changed from 1.97 to 1.9		VL:1.97		AC:0.265 (0.002)
Epoch 12 (0.454 sec): loss changed from 1.94 to 1.86		VL

KeyboardInterrupt: 

In [167]:
valid[:10]

array([[2, 5, 2, 5, 1, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 2, 2, 1, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [5, 4, 2, 3, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 2, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 3, 1, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 1, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 5, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 3, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 4, 3, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [99]:
type((1,2))

tuple